# Problem Statement
1. **Spectra's Objective**: The sports magazine wants to repurpose unused cricket images for social media posts under the #throwback category to boost engagement during off-seasons.

2. **Engagement Strategy**: These posts aim to drive interaction on social media by filling the content gap when there are no live updates to share.

3. **Image Analyser System**: Develop an AG2-based image analyzer that identifies the sport, analyzes team dynamics and winning probabilities, and generates engaging commentary for the images.

1. https://i.ytimg.com/vi/CarUZqvtzeU/maxresdefault.jpg
2. https://i.ytimg.com/vi/SAuwTBsRMcg/maxresdefault.jpg

In [ ]:
#!pip install Pillow

In [1]:
import os 
import warnings
warnings.filterwarnings("ignore")
import autogen
from autogen import ConversableAgent
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from IPython.display import Markdown, display, Image

In [2]:
# load env variables
from dotenv import load_dotenv
load_dotenv('/Users/admin/Desktop/AutoGen/Module 1/.env')

True

In [3]:
# configuration for LLM
config_list_1 = {
    "config_list": [{"model": "gpt-4o", "temperature": 0.3}]
}

In [4]:
# configuration for LLM
config_list_2 = {
    "config_list": [{"model": "gpt-4o", "temperature": 0.5}]
}

In [5]:
game_analyzer_agent = MultimodalConversableAgent(
    "Game_Analyzer_Agent",
    system_message = """
You are tasked with analyzing a sports image to determine the game being played. 
Identify the type of sport (e.g., football, basketball, soccer,cricket or any other) 
from the image and recognize the teams that are involved. 
Provide basic details such as the names of players and name of the teams and the current 
state of the game (e.g., score, quarter, half-time status, overs reaminnig etc.). 
If there are any visible details that provide additional details about the game, mention them as well. 
""",
    llm_config=config_list_2,
    human_input_mode="NEVER",
)


In [6]:
probability_agent = ConversableAgent(
    "Probability_Analyzer_Agent",
    system_message = """
You are tasked with calculating the winning probability for each team based on the current score and the score they need to chase. 
Search across the web to collect information about the match and Use the game stats and their historical performences and 
calculate the probability of each team winning based on their performance up until now. Consider factors such as the score gap, time left in the game, 
and give the approximate percentage of each teams for wining the game.
""",
    llm_config= config_list_1,
    human_input_mode='NEVER',
)

In [7]:
commentory_agent = MultimodalConversableAgent(
    "Commentory_agent",
    system_message = """
You are tasked with delivering a detailed, real-time commentary on the game:
- Take help of the Game Analyzer Agent's findings (game type, score, teams, etc.).
- Use the Probability Agent's calculations to provide insight into the game's likely outcomes.
Focus on creating a seamless and detailed narrative that provides key moments, highlights, and ongoing excitement for fans.
""",
    llm_config= config_list_2,
    human_input_mode="NEVER",
)

In [8]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="""
    You are assisting in analyzing sports images and coordinating among agents to extract relevant game details. If the input contains URLs, fetch and incorporate relevant details.
""",
    human_input_mode="TERMINATE",  # Try between ALWAYS, NEVER, and TERMINATE
    max_consecutive_auto_reply=0,
    code_execution_config={
        "use_docker": False  # Set to True if Docker is available
    },
)

In [9]:
group_meet = autogen.GroupChat(
    agents=[user_proxy,game_analyzer_agent, probability_agent, commentory_agent],
    messages=[],
    max_round=4,
    speaker_selection_method= "auto",
    select_speaker_auto_llm_config = config_list_1
)
group_manager = autogen.GroupChatManager(
    groupchat=group_meet,
)

In [10]:
meeting = user_proxy.initiate_chat(
    group_manager,
    message="""Analyze the sports image  and extract the game type or  teams involved or any visible players or score
    <img https://i.ytimg.com/vi/CarUZqvtzeU/maxresdefault.jpg>""",
summary_method="last_msg",
)

User_proxy (to chat_manager):

Analyze the sports image  and extract the game type or  teams involved or any visible players or score
    <img https://i.ytimg.com/vi/CarUZqvtzeU/maxresdefault.jpg>

--------------------------------------------------------------------------------

Next speaker: Game_Analyzer_Agent

Game_Analyzer_Agent (to chat_manager):

The image is from a cricket match between India and South Africa. 

- **Player:** Ishan Kishan, who scored 76 runs off 48 balls.
- **Score:** India is at 137-1 after 12.5 overs.
- **Bowler:** Maharaj, with figures of 0-43 in 2.5 overs.
- **Other Player:** Shreyas, who has scored 28 runs off 15 balls.
- **Fall of Wicket:** 137-2, with Ishan Kishan caught by Stubbs off Maharaj.

--------------------------------------------------------------------------------

Next speaker: Probability_Analyzer_Agent

Probability_Analyzer_Agent (to chat_manager):

Based on the information extracted from the image, we can analyze the current match situation 

In [11]:
agent_name_to_find = ["Game_Analyzer_Agent", "Probability_Analyzer_Agent", "Commentory_agent"]

display(Image(url="https://i.ytimg.com/vi/CarUZqvtzeU/maxresdefault.jpg", width = 600))
for agent_name in agent_name_to_find:

    # Iterate through the dictionary to find the last conversation for the specified agent
    for agent, interactions in group_manager.chat_messages.items():
        # Filter interactions by agent name
        filtered_interactions = [i for i in interactions if i.get("name") == agent_name]
        if filtered_interactions:
            # Print the last conversation's content
            # print(filtered_interactions[-1]["content"])
            display(Markdown('## ' + agent_name))
            display(Markdown(filtered_interactions[-1]["content"]))
            break
    else:
        print("Agent not found or no interactions available.")


## Game_Analyzer_Agent

The image is from a cricket match between India and South Africa. 

- **Player:** Ishan Kishan, who scored 76 runs off 48 balls.
- **Score:** India is at 137-1 after 12.5 overs.
- **Bowler:** Maharaj, with figures of 0-43 in 2.5 overs.
- **Other Player:** Shreyas, who has scored 28 runs off 15 balls.
- **Fall of Wicket:** 137-2, with Ishan Kishan caught by Stubbs off Maharaj.

## Probability_Analyzer_Agent

Based on the information extracted from the image, we can analyze the current match situation and calculate the winning probabilities for each team:

### Current Match Situation:
- **Teams:** India vs. South Africa
- **Current Score:** India is 137/2 in 12.5 overs.
- **Key Players:** Ishan Kishan (76 off 48 balls), Shreyas (28 off 15 balls)
- **Bowler:** Maharaj (0-43 in 2.5 overs)

### Factors to Consider:
1. **Score Gap:** India is currently at a strong position with a high run rate, having lost only 2 wickets.
2. **Overs Remaining:** 7.1 overs remaining in the innings.
3. **Historical Performance:** India has a strong batting lineup and has performed well in chasing or setting high scores in T20 matches.
4. **Pitch Conditions:** Assuming the pitch is favorable for batting given the high score rate.
5. **Bowling Attack:** South Africa's bowling figures suggest they are struggling to contain runs.

### Winning Probability Calculation:
- **India's Probability:**
  - Given the strong start and high run rate, India's probability of winning is high. They have a deep batting lineup and are in a commanding position.
  - **Approximate Probability:** 75%

- **South Africa's Probability:**
  - South Africa needs to take quick wickets and contain runs to have a chance of winning. Their current bowling performance is not favorable.
  - **Approximate Probability:** 25%

These probabilities are approximate and based on the current match situation and historical performances. The actual outcome may vary based on real-time developments in the game.

## Commentory_agent

### Match Analysis

#### Current Situation:
- **Teams:** India vs. South Africa
- **Score:** India at 137/2 in 12.5 overs

#### Key Players:
- **Ishan Kishan:** Scored 76 off 48 balls, contributing significantly to India's strong position.
- **Shreyas:** Currently at 28 off 15 balls, maintaining the momentum.

#### Bowling:
- **Maharaj:** Has conceded 43 runs in 2.5 overs without taking a wicket.

### Game Dynamics:
- **India's Position:** With a high run rate and only 2 wickets down, India is in a dominant position.
- **South Africa's Strategy:** Needs to take quick wickets and restrict runs to shift the game's momentum.

### Winning Probabilities:
- **India:** Approximately 75% chance of winning due to their strong start and batting depth.
- **South Africa:** Approximately 25% chance, needing a significant turnaround in bowling performance.

Overall, India is well-placed to capitalize on their current advantage, while South Africa needs strategic breakthroughs to challenge India's lead.

In [12]:
meeting1 = user_proxy.initiate_chat(
    group_manager,
    message="""Analyze the sports image  and extract the game type or  teams involved or any visible players or score
    <img https://i.ytimg.com/vi/SAuwTBsRMcg/maxresdefault.jpg>.""",
summary_method="last_msg",
)

User_proxy (to chat_manager):

Analyze the sports image  and extract the game type or  teams involved or any visible players or score
    <img https://i.ytimg.com/vi/SAuwTBsRMcg/maxresdefault.jpg>.

--------------------------------------------------------------------------------

Next speaker: Game_Analyzer_Agent

Game_Analyzer_Agent (to chat_manager):

The image is from a cricket match in the Indian Premier League (IPL). The teams involved are Royal Challengers Bangalore (RCB) and Chennai Super Kings (CSK). The score for RCB is 198-7 after 18.2 overs. The batsmen at the crease are Hasaranga (1 run) and Prabhudessai (1 run). RCB needs 29 runs from 10 balls to win.

--------------------------------------------------------------------------------

Next speaker: Probability_Analyzer_Agent

Probability_Analyzer_Agent (to chat_manager):

To calculate the winning probability for each team, let's consider the following factors:

1. **Current Score and Target**: 
   - RCB is at 198/7 in 18.2 o

In [13]:
agent_name_to_find = ["Game_Analyzer_Agent", "Probability_Analyzer_Agent", "Commentory_agent"]

display(Image(url="https://i.ytimg.com/vi/SAuwTBsRMcg/maxresdefault.jpg", width = 600))

for agent_name in agent_name_to_find:

    # Iterate through the dictionary to find the last conversation for the specified agent
    for agent, interactions in group_manager.chat_messages.items():
        # Filter interactions by agent name
        filtered_interactions = [i for i in interactions if i.get("name") == agent_name]
        if filtered_interactions:
            # Print the last conversation's content
            # print(filtered_interactions[-1]["content"])
            display(Markdown('## ' + agent_name))
            display(Markdown(filtered_interactions[-1]["content"]))
            break
    else:
        print("Agent not found or no interactions available.")


## Game_Analyzer_Agent

The image is from a cricket match in the Indian Premier League (IPL). The teams involved are Royal Challengers Bangalore (RCB) and Chennai Super Kings (CSK). The score for RCB is 198-7 after 18.2 overs. The batsmen at the crease are Hasaranga (1 run) and Prabhudessai (1 run). RCB needs 29 runs from 10 balls to win.

## Probability_Analyzer_Agent

To calculate the winning probability for each team, let's consider the following factors:

1. **Current Score and Target**: 
   - RCB is at 198/7 in 18.2 overs, chasing a target set by CSK.
   - RCB needs 29 runs from 10 balls to win.

2. **Score Gap**: 
   - RCB needs 29 runs with only 10 balls remaining, which is a challenging task.

3. **Wickets in Hand**: 
   - RCB has 3 wickets remaining, which means they have limited batting resources left.

4. **Historical Performance**: 
   - CSK is known for their strong bowling attack, especially in the death overs.
   - RCB's lower order is not known for consistent big hitting.

5. **Current Batsmen**:
   - Hasaranga and Prabhudessai are at the crease. Neither is known for finishing games with big scores, which adds to the difficulty for RCB.

6. **Time Left in the Game**: 
   - With only 10 balls remaining, the required run rate is very high.

Based on these factors, let's estimate the winning probabilities:

- **CSK's Winning Probability**: Given the high required run rate, the strong bowling attack of CSK, and the fact that RCB's current batsmen are not known for finishing games, CSK has a higher probability of winning. I would estimate CSK's winning probability to be around 85%.

- **RCB's Winning Probability**: RCB has a slim chance of winning due to the high required run rate and the pressure on the lower-order batsmen. I would estimate RCB's winning probability to be around 15%.

These probabilities are approximate and based on the current match situation and historical performances.

## Commentory_agent

The image shows an intense IPL cricket match between Royal Challengers Bangalore (RCB) and Chennai Super Kings (CSK). RCB is chasing a target, currently at 198-7 in 18.2 overs, needing 29 runs from 10 balls to win. The players at the crease are Hasaranga and Prabhudessai, both contributing minimally so far. Given the high required run rate and CSK's strong bowling, RCB faces a challenging task to secure victory.